In [1]:
import pandas as pd
import numpy as np
import time
from datetime import datetime
import geopandas as gp
import matplotlib
import matplotlib.pyplot as plt
import os
from shapely.geometry import Point
from tqdm import tqdm
from tqdm import trange

%matplotlib inline

In [2]:
## BASE ANALITICA

In [3]:
# URL do arquivo shapefile
url = 'https://geoftp.ibge.gov.br/organizacao_do_territorio/malhas_territoriais/malhas_municipais/municipio_2022/Brasil/BR/BR_Municipios_2022.zip'

# Lendo o arquivo shapefile
geo_brasil = gp.read_file(url)

geo_brasil = gp.GeoDataFrame(geo_brasil)
geo_brasil = geo_brasil.set_geometry('geometry')
# Visualizando as primeiras linhas do dataframe
print(geo_brasil.head())

    CD_MUN                 NM_MUN SIGLA_UF  AREA_KM2  \
0  1100015  Alta Floresta D'Oeste       RO  7067.127   
1  1100023              Ariquemes       RO  4426.571   
2  1100031                 Cabixi       RO  1314.352   
3  1100049                 Cacoal       RO  3793.000   
4  1100056             Cerejeiras       RO  2783.300   

                                            geometry  
0  POLYGON ((-62.00806 -12.13379, -62.00784 -12.2...  
1  POLYGON ((-63.17933 -10.13924, -63.17746 -10.1...  
2  POLYGON ((-60.52408 -13.32137, -60.37162 -13.3...  
3  POLYGON ((-61.35502 -11.50452, -61.35524 -11.5...  
4  POLYGON ((-60.82135 -13.11910, -60.81773 -13.1...  


In [4]:
# Verificando se há dados duplicados em qualquer coluna
if geo_brasil.duplicated().any():
    # Se houver, imprime os dados duplicados
    print('Dados duplicados:')
    print(geo_brasil[geo_brasil.duplicated()])
else:
    # Caso contrário, imprime a mensagem informando que não há dados duplicados
    print('Não há dados duplicados.')

Não há dados duplicados.


In [5]:
# IMPORTA TRUSTED MOBILE E DELETA DADOS QUE NÃO SERÃO USADOS NA BASE ANALITICA
trusted_mobile = gp.read_file('C:/Users/crash/Desktop/trusted_speedtest_mobile.shp')
trusted_mobile.head()

,quadkey,avg_d_kbps,avg_u_kbps,avg_lat_ms,tests,devices,index_righ,CD_MUN,latitude,longitude,date,quarter,network_ty,geometry
0,0323230233223102,15591,17249,26,4,2,138,1400100,2.847033,-60.751648,2022-01-01,1,mobile,"POLYGON ((-60.75439 2.84978, -60.74890 2.84978..."
1,0323230233232230,16998,20375,25,2,1,138,1400100,2.819601,-60.718689,2022-01-01,1,mobile,"POLYGON ((-60.72144 2.82234, -60.71594 2.82234..."
2,0323232011001023,17268,8038,29,2,2,138,1400100,2.792168,-60.768127,2022-01-01,1,mobile,"POLYGON ((-60.77087 2.79491, -60.76538 2.79491..."
3,0323230233232011,116923,4563,36,1,1,138,1400100,2.852520,-60.713196,2022-01-01,1,mobile,"POLYGON ((-60.71594 2.85526, -60.71045 2.85526..."
4,0323230233232033,5092,7704,31,2,2,138,1400100,2.836060,-60.713196,2022-01-01,1,mobile,"POLYGON ((-60.71594 2.83880, -60.71045 2.83880..."


In [6]:
# IMPORTA TRUSTED MOBILE E DELETA DADOS QUE NÃO SERÃO USADOS NA BASE ANALITICA
trusted_fixed = gp.read_file('C:/Users/crash/Desktop/trusted_speedtest_fixed.shp')
trusted_fixed.head()

,quadkey,avg_d_kbps,avg_u_kbps,avg_lat_ms,tests,devices,index_righ,CD_MUN,latitude,longitude,date,quarter,network_ty,geometry
0,2110223123011000,157341,56947,13,1095,259,3174,3205309,-20.305993,-40.295105,2022-01-01,1,fixed,"POLYGON ((-40.29785 -20.30342, -40.29236 -20.3..."
1,2110223121232032,184049,107142,4,120,45,3174,3205309,-20.280232,-40.328064,2022-01-01,1,fixed,"POLYGON ((-40.33081 -20.27766, -40.32532 -20.2..."
2,2110223123001110,176971,90063,23,221,52,3174,3205309,-20.305993,-40.350037,2022-01-01,1,fixed,"POLYGON ((-40.35278 -20.30342, -40.34729 -20.3..."
3,2110223121223330,160729,94449,5,105,26,3174,3205309,-20.295690,-40.350037,2022-01-01,1,fixed,"POLYGON ((-40.35278 -20.29311, -40.34729 -20.2..."
4,2110223121233110,167756,63182,9,590,119,3174,3205309,-20.264774,-40.262146,2022-01-01,1,fixed,"POLYGON ((-40.26489 -20.26220, -40.25940 -20.2..."


In [7]:
trusted_fixed.shape

(1807570, 14)

In [8]:
crs = trusted_fixed.crs
print(crs)

EPSG:4674


In [9]:
trusted_mobile.shape

(588373, 14)

In [10]:
geo_brasil.shape

(5572, 5)

In [11]:
crs = trusted_mobile.crs
print(crs)

EPSG:4674


In [12]:
crs = trusted_fixed.crs
print(crs)

EPSG:4674


In [13]:
crs = geo_brasil.crs
print(crs)

EPSG:4674


In [14]:
# filtra dados apenas do Brasil
# trusted_mobile_2 = trusted_mobile.to_crs(geo_brasil.crs)
# trusted_mobile_3 = gp.sjoin(trusted_mobile_2, geo_brasil, how="left", predicate='intersects')

In [15]:
geo_brasil = geo_brasil.to_crs(epsg=4674)
trusted_mobile = trusted_mobile.to_crs(epsg=4674)
trusted_fixed = trusted_fixed.to_crs(epsg=4674)


In [16]:
# trusted_mobile_2 = trusted_mobile.to_crs(geo_brasil.crs)
# trusted_mobile_3 = gp.sjoin(trusted_mobile_raw, geo_brasil, how="inner", predicate='intersects')
# trusted_mobile_3 = trusted_mobile_3.drop_duplicates(subset='geometry')
df_merged = pd.merge(trusted_mobile, geo_brasil, on='CD_MUN', how='inner')
df_merged_fixed = pd.merge(trusted_fixed, geo_brasil, on='CD_MUN', how='inner')


In [17]:
df_merged.head()

,quadkey,avg_d_kbps,avg_u_kbps,avg_lat_ms,tests,devices,index_righ,CD_MUN,latitude,longitude,date,quarter,network_ty,geometry_x,NM_MUN,SIGLA_UF,AREA_KM2,geometry_y
0,0323230233223102,15591,17249,26,4,2,138,1400100,2.847033,-60.751648,2022-01-01,1,mobile,"POLYGON ((-60.75439 2.84978, -60.74890 2.84978...",Boa Vista,RR,5687.037,"POLYGON ((-60.67654 3.46012, -60.67239 3.45866..."
1,0323230233232230,16998,20375,25,2,1,138,1400100,2.819601,-60.718689,2022-01-01,1,mobile,"POLYGON ((-60.72144 2.82234, -60.71594 2.82234...",Boa Vista,RR,5687.037,"POLYGON ((-60.67654 3.46012, -60.67239 3.45866..."
2,0323232011001023,17268,8038,29,2,2,138,1400100,2.792168,-60.768127,2022-01-01,1,mobile,"POLYGON ((-60.77087 2.79491, -60.76538 2.79491...",Boa Vista,RR,5687.037,"POLYGON ((-60.67654 3.46012, -60.67239 3.45866..."
3,0323230233232011,116923,4563,36,1,1,138,1400100,2.852520,-60.713196,2022-01-01,1,mobile,"POLYGON ((-60.71594 2.85526, -60.71045 2.85526...",Boa Vista,RR,5687.037,"POLYGON ((-60.67654 3.46012, -60.67239 3.45866..."
4,0323230233232033,5092,7704,31,2,2,138,1400100,2.836060,-60.713196,2022-01-01,1,mobile,"POLYGON ((-60.71594 2.83880, -60.71045 2.83880...",Boa Vista,RR,5687.037,"POLYGON ((-60.67654 3.46012, -60.67239 3.45866..."


In [18]:
df_merged_fixed.head()

,quadkey,avg_d_kbps,avg_u_kbps,avg_lat_ms,tests,devices,index_righ,CD_MUN,latitude,longitude,date,quarter,network_ty,geometry_x,NM_MUN,SIGLA_UF,AREA_KM2,geometry_y
0,2110223123011000,157341,56947,13,1095,259,3174,3205309,-20.305993,-40.295105,2022-01-01,1,fixed,"POLYGON ((-40.29785 -20.30342, -40.29236 -20.3...",Vitória,ES,97.123,"MULTIPOLYGON (((-40.29020 -20.28757, -40.29024..."
1,2110223121232032,184049,107142,4,120,45,3174,3205309,-20.280232,-40.328064,2022-01-01,1,fixed,"POLYGON ((-40.33081 -20.27766, -40.32532 -20.2...",Vitória,ES,97.123,"MULTIPOLYGON (((-40.29020 -20.28757, -40.29024..."
2,2110223123001110,176971,90063,23,221,52,3174,3205309,-20.305993,-40.350037,2022-01-01,1,fixed,"POLYGON ((-40.35278 -20.30342, -40.34729 -20.3...",Vitória,ES,97.123,"MULTIPOLYGON (((-40.29020 -20.28757, -40.29024..."
3,2110223121223330,160729,94449,5,105,26,3174,3205309,-20.295690,-40.350037,2022-01-01,1,fixed,"POLYGON ((-40.35278 -20.29311, -40.34729 -20.2...",Vitória,ES,97.123,"MULTIPOLYGON (((-40.29020 -20.28757, -40.29024..."
4,2110223121233110,167756,63182,9,590,119,3174,3205309,-20.264774,-40.262146,2022-01-01,1,fixed,"POLYGON ((-40.26489 -20.26220, -40.25940 -20.2...",Vitória,ES,97.123,"MULTIPOLYGON (((-40.29020 -20.28757, -40.29024..."


In [19]:
df_merged.shape

(588373, 18)

In [20]:
df_merged_fixed.shape

(1807570, 18)

In [21]:
df_merged = df_merged[['date','quarter','latitude','longitude', 'network_ty', 'NM_MUN','SIGLA_UF','AREA_KM2','devices','tests','avg_lat_ms','avg_u_kbps','avg_d_kbps']]
df_merged_fixed = df_merged_fixed[['date','quarter','latitude','longitude','network_ty', 'NM_MUN','SIGLA_UF','AREA_KM2','devices','tests','avg_lat_ms','avg_u_kbps','avg_d_kbps']]


In [22]:
df_merged.head()

,date,quarter,latitude,longitude,network_ty,NM_MUN,SIGLA_UF,AREA_KM2,devices,tests,avg_lat_ms,avg_u_kbps,avg_d_kbps
0,2022-01-01,1,2.847033,-60.751648,mobile,Boa Vista,RR,5687.037,2,4,26,17249,15591
1,2022-01-01,1,2.819601,-60.718689,mobile,Boa Vista,RR,5687.037,1,2,25,20375,16998
2,2022-01-01,1,2.792168,-60.768127,mobile,Boa Vista,RR,5687.037,2,2,29,8038,17268
3,2022-01-01,1,2.852520,-60.713196,mobile,Boa Vista,RR,5687.037,1,1,36,4563,116923
4,2022-01-01,1,2.836060,-60.713196,mobile,Boa Vista,RR,5687.037,2,2,31,7704,5092


In [23]:
df_merged_fixed.head()

,date,quarter,latitude,longitude,network_ty,NM_MUN,SIGLA_UF,AREA_KM2,devices,tests,avg_lat_ms,avg_u_kbps,avg_d_kbps
0,2022-01-01,1,-20.305993,-40.295105,fixed,Vitória,ES,97.123,259,1095,13,56947,157341
1,2022-01-01,1,-20.280232,-40.328064,fixed,Vitória,ES,97.123,45,120,4,107142,184049
2,2022-01-01,1,-20.305993,-40.350037,fixed,Vitória,ES,97.123,52,221,23,90063,176971
3,2022-01-01,1,-20.295690,-40.350037,fixed,Vitória,ES,97.123,26,105,5,94449,160729
4,2022-01-01,1,-20.264774,-40.262146,fixed,Vitória,ES,97.123,119,590,9,63182,167756


In [24]:
# todos os campos numericos já são do tipo inteiro, com execção do kilometro

In [25]:
# df_fixed.head()

In [26]:
# df_fixed.shape

In [27]:
# df_fixed = df_fixed[['date', 'network_ty', 'NM_MUN','SIGLA_UF','AREA_KM2','devices','tests','avg_lat_ms','avg_u_kbps','avg_d_kbps']]


In [28]:
# df_fixed.head()

In [29]:
df_merged = df_merged.append(df_merged_fixed)


C:\Users\crash\AppData\Local\Temp\ipykernel_5180\1402631800.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_merged = df_merged.append(df_merged_fixed)


In [30]:
df_merged.head()

,date,quarter,latitude,longitude,network_ty,NM_MUN,SIGLA_UF,AREA_KM2,devices,tests,avg_lat_ms,avg_u_kbps,avg_d_kbps
0,2022-01-01,1,2.847033,-60.751648,mobile,Boa Vista,RR,5687.037,2,4,26,17249,15591
1,2022-01-01,1,2.819601,-60.718689,mobile,Boa Vista,RR,5687.037,1,2,25,20375,16998
2,2022-01-01,1,2.792168,-60.768127,mobile,Boa Vista,RR,5687.037,2,2,29,8038,17268
3,2022-01-01,1,2.852520,-60.713196,mobile,Boa Vista,RR,5687.037,1,1,36,4563,116923
4,2022-01-01,1,2.836060,-60.713196,mobile,Boa Vista,RR,5687.037,2,2,31,7704,5092


In [31]:
df_merged.shape

(2395943, 13)

In [32]:
# Agrupar pelos campos de dimensão e calcular a soma das colunas numéricas
df_soma = df_merged.groupby(['latitude','longitude','date','quarter', 'network_ty', 'NM_MUN', 'SIGLA_UF', 'AREA_KM2']).agg({'devices': 'sum', 'tests': 'sum', 'avg_lat_ms': 'sum', 'avg_u_kbps': 'sum', 'avg_d_kbps': 'sum'}).reset_index()

# Exibir o DataFrame resultante
print(df_soma)

          latitude  longitude        date quarter network_ty  \
0       -33.749464 -53.385315  2022-01-01       1      fixed   
1       -33.749464 -53.385315  2022-10-01       4     mobile   
2       -33.749464 -53.385315  2023-01-01       1      fixed   
3       -33.749464 -53.385315  2023-01-01       1     mobile   
4       -33.749464 -53.379822  2022-01-01       1      fixed   
...            ...        ...         ...     ...        ...   
2395938   4.595589 -60.163879  2022-07-01       3      fixed   
2395939   4.595589 -60.163879  2022-10-01       4      fixed   
2395940   4.595589 -60.163879  2023-01-01       1      fixed   
2395941   4.595589 -60.158386  2022-04-01       2      fixed   
2395942   4.595589 -60.158386  2022-10-01       4      fixed   

                          NM_MUN SIGLA_UF  AREA_KM2  devices  tests  \
0        Santa Vitória do Palmar       RS  5206.977        2      5   
1        Santa Vitória do Palmar       RS  5206.977        1      1   
2        Santa Vit

In [33]:
df_soma.shape

(2395943, 13)

In [34]:
df_soma.to_csv('base_analitica_speedtest.csv', sep=';', index=False)